# Analyzing a dataset and performing a classification analysis

## Import our libraries

In [1]:
#Import standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import random
%matplotlib inline

In [3]:
#Import modules specific to the machine learning aspect of this analysis
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#Import classification algorithm modules
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.mixture import GaussianMixture

## Read the data

In [4]:
df = pd.read_csv('bank.csv')

In [7]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


The objective in this exercise is to analyze the data, extract some meaningful insights about the features and their relationship to the outcome (deposit = yes or no), come up with some strategies for future campaigns, and build a predictive model that will allow us in the future to determine whether or not a customer will subscribe (deposit = yes)

**Brief explanation of the "not so intuitive" features**

Housing - has a house loan

Loan - has a personal loan

Month - last contact month

Day - last contact day

Duartion - last contact duration (phone call)

Campaign - number of contacts performed during this campaign and for this client

Pdays - number of days since **previous** contact

Poutcome - outcome of **previous** marketing campaign and for this client

Deposit - this is the dependent variable that we are going to want to build a predictive model to classify, describes whether or not a client subscribed (deposit = yes) - it shows whether or not the previous marketing campaign was successful for this client